In [1]:
import pandas as pd
from pandas import merge
import numpy as np
from matplotlib import pyplot
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.font_manager
import seaborn as sns
%matplotlib inline
plt.style.use('ggplot')
from sklearn import svm

In [2]:
import glob,os

files = glob.glob('data/*.csv')

In [3]:
df = pd.concat([pd.read_csv(f).assign(f_name=os.path.basename(f).split('t_')[1].split('.')[0]) for f in files])

In [4]:
df.drop('Unnamed: 0', axis=1, inplace=True)

In [5]:
df = df.rename(columns={'time':'weld_time','name':'weld_point','f_name':'robot_name'})
df.head()

weld_time  current  voltage  weld_point robot_name
0        434     8403     2059           1   TC_112_1
1        453     8380     2030           2   TC_112_1
2        453     8390     2041           3   TC_112_1
3        451     8382     2003           4   TC_112_1
4        288     8154     1524           5   TC_112_1

In [6]:
df = df[df['current']>=2000]

In [7]:
export_csv = df.to_csv(r'df_weld_190923.csv',index=False,header=True,sep='\t', encoding='utf-8')

In [7]:
lyr2_1mm_abno = pd.read_csv('2layer_1mm_abno_condition.csv',sep = '\t')

In [8]:
lyr2_1mm_abno.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 25 columns):
Consecutive archive number                       9 non-null int64
Name of the component                            0 non-null float64
Module name                                      9 non-null object
Date of the welding                              9 non-null object
Time of the welding                              9 non-null object
Spot counter                                     9 non-null int64
Current program                                  9 non-null int64
Component number of the produced component       0 non-null float64
Overall welding status of the component          0 non-null float64
Current spot                                     9 non-null int64
Welding spot status                              9 non-null object
Gun name                                         9 non-null object
Current gun                                      9 non-null int64
Average act. current          

In [9]:
lyr2_1mm_abno = pd.read_csv('2layer_1mm_abno_condition.csv',sep = '\t')
lyr2_2mm_abno = pd.read_csv('2layer_2mm_abno_condition.csv',sep = '\t')
lyr2_slr_abno = pd.read_csv('2layer_sealer_abno_condition.csv',sep = '\t')
lyr3_1mm_abno = pd.read_csv('3layer_1mm_abno_condition.csv',sep = '\t')
lyr3_2mm_abno = pd.read_csv('3layer_2mm_abno_condition.csv',sep = '\t')
lyr3_slr_abno = pd.read_csv('3layer_sealer_abno_condition.csv',sep = '\t')
lyr4_2mm_abno = pd.read_csv('4layer_1mm_abno_condition.csv',sep = '\t')
lyr4_2mm_abno = pd.read_csv('4layer_2mm_abno_condition.csv',sep = '\t')
lyr4_slr_abno = pd.read_csv('4layer_sealer_abno_condition.csv',sep = '\t')

lyr2_1mm_abno['condition'] = 'lyr2_1mm_abno'
lyr2_2mm_abno['condition'] = 'lyr2_2mm_abno'
lyr2_slr_abno['condition'] = 'lyr2_slr_abno'
lyr3_1mm_abno['condition'] = 'lyr3_1mm_abno'
lyr3_2mm_abno['condition'] = 'lyr3_2mm_abno'
lyr3_slr_abno['condition'] = 'lyr3_slr_abno'
lyr4_2mm_abno['condition'] = 'lyr4_2mm_abno'
lyr4_2mm_abno['condition'] = 'lyr4_2mm_abno'
lyr4_slr_abno['condition'] = 'lyr4_slr_abno'

df_abno = pd.concat([lyr2_1mm_abno,lyr2_2mm_abno,lyr2_slr_abno,lyr3_1mm_abno,lyr3_2mm_abno,
                     lyr3_slr_abno,lyr4_2mm_abno,lyr4_2mm_abno,lyr4_slr_abno,])

df_abno.reset_index(drop=True)

df1 = df_abno[['Average act. current','Average actual voltage of electrodes',
               'Actual current time','Energy','condition']]

df1 = df1.rename(columns = {'Average act. current':'current',
                            'Average actual voltage of electrodes':'voltage',
                            'Actual current time':'weld_time'})

In [11]:
len(df['current'])

9938661

In [10]:
len(df1['current'])

86

In [12]:
X_outliers = df1['current']

In [13]:
lyr2_nor = pd.read_csv('2layer_normal_condition.csv',sep = '\t')
lyr3_nor = pd.read_csv('3layer_normal_condition.csv',sep = '\t')
lyr4_nor = pd.read_csv('4layer_normal_condition.csv',sep = '\t')

lyr2_nor['condition'] = 'lyr2_nor'
lyr3_nor['condition'] = 'lyr3_nor'
lyr4_nor['condition'] = 'lyr4_nor'

df_nor = pd.concat([lyr2_nor,lyr3_nor,lyr4_nor])

df_nor.reset_index(drop=True)

df2 = df_nor[['Average act. current','Average actual voltage of electrodes',
               'Actual current time','Energy','condition']]

df2 = df2.rename(columns = {'Average act. current':'current','Average actual voltage of electrodes':'voltage',
                            'Actual current time':'weld_time'})

X_normal = df2['current']

In [21]:
a = round(len(df)*0.003)

In [22]:
df_tr = df.iloc[:a,1]

In [26]:
df_tr.values

array([8403, 8380, 8390, ..., 8324, 6427, 6482], dtype=int64)

In [23]:
import random

In [27]:
from random import shuffle
M = df_tr.values
shuffle(M)
print (M)
X_df = pd.DataFrame({'M':M})

M = df_tr.values
shuffle(M)
print (M)
X_df['N'] = M

X_df.head()

[5996 7758 8184 ... 6421 8347 7888]
[6263 8152 7758 ... 8388 8151 8185]


M     N
0  5996  6263
1  7758  8152
2  8184  7758
3  8400  6240
4  6612  8369

In [28]:
X_train = X_df.values*0.0001

In [29]:
X_train

array([[0.5996, 0.6263],
       [0.7758, 0.8152],
       [0.8184, 0.7758],
       ...,
       [0.6421, 0.8388],
       [0.8347, 0.8151],
       [0.7888, 0.8185]])

In [30]:
from random import shuffle
M = df_tr.values
shuffle(M)
print(M)
X_df2 = pd.DataFrame({'M':M})

M = df_tr.values
shuffle(M)
print(M)
X_df2['N'] = M

X_df2.head()

[8426 7844 6608 ... 8388 8383 8482]
[7748 7768 7892 ... 8384 7840 8411]


M     N
0  8426  7748
1  7844  7768
2  6608  7892
3  8212  8401
4  7776  8303

In [31]:
X_test = X_df2.values*0.0001

In [32]:
X_test

array([[0.8426, 0.7748],
       [0.7844, 0.7768],
       [0.6608, 0.7892],
       ...,
       [0.8388, 0.8384],
       [0.8383, 0.784 ],
       [0.8482, 0.8411]])

In [33]:
xx, yy = np.meshgrid(np.linspace(0,1,500),np.linspace(0,1,500))

In [34]:
X_outlier_value = X_outliers.values * 0.0001

In [49]:
X_outlier_value

array([0.7147, 0.7181, 0.7175, 0.72  , 0.7187, 0.7171, 0.718 , 0.7168,
       0.7178, 0.7117, 0.717 , 0.7184, 0.7161, 0.7145, 0.7162, 0.7179,
       0.7106, 0.717 , 0.7158, 0.7196, 0.7159, 0.7179, 0.7166, 0.7063,
       0.7047, 0.7081, 0.714 , 0.7184, 0.7114, 0.7142, 0.7116, 0.7173,
       0.7118, 0.7111, 0.7175, 0.7113, 0.721 , 0.7115, 0.7143, 0.7119,
       0.7062, 0.7023, 0.7125, 0.7114, 0.7083, 0.7119, 0.7101, 0.7089,
       0.7175, 0.7119, 0.716 , 0.7148, 0.7121, 0.7196, 0.7132, 0.6995,
       0.7194, 0.715 , 0.7132, 0.7038, 0.7059, 0.7143, 0.706 , 0.7058,
       0.7035, 0.7078, 0.7194, 0.715 , 0.7132, 0.7038, 0.7059, 0.7143,
       0.706 , 0.7058, 0.7035, 0.7078, 0.7057, 0.7051, 0.7137, 0.7131,
       0.7144, 0.7046, 0.7153, 0.7134, 0.7163, 0.7055])

In [38]:
len(X_outlier_value)

86

In [61]:
X_outlier_value = X_outlier_value.reshape(X_outlier_value.shape[0],1)

In [59]:
from random import shuffle
M = X_outlier_value
shuffle(M)
print(M)
out = pd.DataFrame({'M':M})

M = X_outlier_value
shuffle(M)
print(M)
out['N'] = M

out.head()

AttributeError: 'numpy.ndarray' object has no attribute 'values'

In [53]:
# fit the model
clf = svm.OneClassSVM(nu=0.1,kernel="rbf",gamma=0.1)

In [54]:
clf

OneClassSVM(cache_size=200, coef0=0.0, degree=3, gamma=0.1, kernel='rbf',
      max_iter=-1, nu=0.1, random_state=None, shrinking=True, tol=0.001,
      verbose=False)

In [55]:
clf.fit(X_train)

OneClassSVM(cache_size=200, coef0=0.0, degree=3, gamma=0.1, kernel='rbf',
      max_iter=-1, nu=0.1, random_state=None, shrinking=True, tol=0.001,
      verbose=False)

In [57]:
y_pred_train = clf.predict(X_train)
y_pred_test = clf.predict(X_test)

In [60]:
y_pred_outliers = clf.predict(X_outlier_value)

ValueError: X.shape[1] = 1 should be equal to 2, the number of features at training time

In [ ]:
y_pred_train

In [ ]:
y_pred_test

In [ ]:
y_pred_outliers

In [ ]:
n_error_train = y_pred_train[y_pred_train == -1].size
n_error_test = y_pred_test[y_pred_test == -1].size
n_error_outliers = y_pred_outliers[y_pred_outliers == 1].size

In [ ]:
# plot the line, the points, and the nearest vectors to the plane
Z = clf.decision_function(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)

In [ ]:
plt.title("Novelty Detection")

plt.contourf(xx, yy, Z, levels = np.linspace(Z.min(),0,7), cmap=plt.cm.PuBu)
a = plt.contour(xx, yy, Z, levels=[0], linewidths=2, colors='darkred')
plt.contourf(xx, yy, Z, levels=[0, Z.max()], colors='palevioletred')

s = 40
b1 = plt.scatter(X_train[:, 0], X_train[:, 1], c ='white', s=s, edgecolors='k')
b2 = plt.scatter(X_test[:, 0], X_test[:, 1], c ='blueviolet', s=s, edgecolors='k')
c = plt.scatter(X_outliers[:, 0], X_outliers[:, 1], c='gold', s=s, edgecolors='k')

plt.axis('tight')
plt.xlim((0, 1))
plt.ylim((0, 1))

plt.legend([a.collections[0], b1, b2, c],
           ["learned frontier", "training observations",
            "new regular observations", "new abnormal observations"],
           loc="upper left",
           prop=matplotlib.font_manager.FontProperties(size=11))

plt.xlabel("error train: %d/200 ; errors novel regular: %d/40 ; "
           "errors novel abnormal: %d/40"
           % (n_error_train, n_error_test, n_error_outliers))

plt.show()

In [ ]:
print(__doc__)

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.font_manager
from sklearn import svm

xx, yy = np.meshgrid(np.linspace(-5, 5, 500), np.linspace(-5, 5, 500))
# Generate train data
X = 0.3 * np.random.randn(100, 2)
X_train = np.r_[X + 2, X - 2]
# Generate some regular novel observations
X = 0.3 * np.random.randn(20, 2)
X_test = np.r_[X + 2, X - 2]
# Generate some abnormal novel observations
X_outliers = np.random.uniform(low=-4, high=4, size=(20, 2))

# fit the model
clf = svm.OneClassSVM(nu=0.1, kernel="rbf", gamma=0.1)
clf.fit(X_train)
y_pred_train = clf.predict(X_train)
y_pred_test = clf.predict(X_test)
y_pred_outliers = clf.predict(X_outliers)
n_error_train = y_pred_train[y_pred_train == -1].size
n_error_test = y_pred_test[y_pred_test == -1].size
n_error_outliers = y_pred_outliers[y_pred_outliers == 1].size

# plot the line, the points, and the nearest vectors to the plane
Z = clf.decision_function(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)

plt.title("Novelty Detection")
plt.contourf(xx, yy, Z, levels=np.linspace(Z.min(), 0, 7), cmap=plt.cm.PuBu)
a = plt.contour(xx, yy, Z, levels=[0], linewidths=2, colors='darkred')
plt.contourf(xx, yy, Z, levels=[0, Z.max()], colors='palevioletred')

s = 40
b1 = plt.scatter(X_train[:, 0], X_train[:, 1], c='white', s=s, edgecolors='k')
b2 = plt.scatter(X_test[:, 0], X_test[:, 1], c='blueviolet', s=s,
                 edgecolors='k')
c = plt.scatter(X_outliers[:, 0], X_outliers[:, 1], c='gold', s=s,
                edgecolors='k')
plt.axis('tight')
plt.xlim((-5, 5))
plt.ylim((-5, 5))
plt.legend([a.collections[0], b1, b2, c],
           ["learned frontier", "training observations",
            "new regular observations", "new abnormal observations"],
           loc="upper left",
           prop=matplotlib.font_manager.FontProperties(size=11))
plt.xlabel(
    "error train: %d/200 ; errors novel regular: %d/40 ; "
    "errors novel abnormal: %d/40"
    % (n_error_train, n_error_test, n_error_outliers))
plt.show()